In [6]:
# Import packages
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
import matplotlib.pylab as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import seaborn as sbn
import warnings
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')

## Part 1: Preparing Data

Run the cells in this part as it is to prepare the data.

Don't change anything unless you get a error.

In [7]:
# Import data
df_main = pd.read_csv('20181207_DataForRegression.csv')
df_main.head()

,NewConn,NewConnRatio,LiveConn,LiveConnRatio,Replacement,Saturday,Maintenance,EntryGain,F,N,...,StationImp,Signal,Structural,Electrical,ConnectionDay,Broken,Shuttle,OriginalConnection,StdRatio,Score
0,3,0.500000,8,1.333333,1,True,1,1,True,False,...,False,True,False,False,9,1,0,6,0.090351,0.049564
1,0,0.000000,6,1.000000,0,True,1,0,True,False,...,False,False,False,False,6,0,0,6,0.082633,0.154406
2,1,0.166667,6,1.000000,1,True,1,1,True,False,...,False,True,False,False,7,1,0,6,0.086103,0.551646
3,1,0.166667,6,1.000000,1,True,1,1,True,False,...,False,True,False,False,7,1,0,6,0.046067,-0.280808
4,1,0.166667,6,1.000000,1,True,1,1,True,False,...,False,True,False,False,7,1,0,6,0.046173,0.218957


In [8]:
# Convert boolean to float
df_main = df_main.apply(lambda x: x.astype(float), axis=0)
df_main.head()

,NewConn,NewConnRatio,LiveConn,LiveConnRatio,Replacement,Saturday,Maintenance,EntryGain,F,N,...,StationImp,Signal,Structural,Electrical,ConnectionDay,Broken,Shuttle,OriginalConnection,StdRatio,Score
0,3.0,0.500000,8.0,1.333333,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,9.0,1.0,0.0,6.0,0.090351,0.049564
1,0.0,0.000000,6.0,1.000000,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,6.0,0.0,0.0,6.0,0.082633,0.154406
2,1.0,0.166667,6.0,1.000000,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,7.0,1.0,0.0,6.0,0.086103,0.551646
3,1.0,0.166667,6.0,1.000000,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,7.0,1.0,0.0,6.0,0.046067,-0.280808
4,1.0,0.166667,6.0,1.000000,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,7.0,1.0,0.0,6.0,0.046173,0.218957


In [66]:
# Define predictors
predictors = list(df_main.columns)
predictors.remove('Score')

# Define Output
output = ['Score']

# Define columns which needs to be scaled
scale_columns = ['NewConn', 'NewConnRatio', 'LiveConn', 'LiveConnRatio', 'Replacement',
                 'Maintenance','EntryGain','ConnectionDay','Broken','Shuttle','OriginalConnection','StdRatio']

# Define a scaler; this will be used later
scaler = StandardScaler()

## Part2: Find the best values for Parameters

1. In the cell below I habve written the code to be used to optimize each paramter.
2. I have commented all parts which does not needs to be changed and parts which needs to be changed.
3. Only change parts where I have mentioned
4. Run this cell for each independent parameter. Example, for random forest first n_estimators, then maxDeth, then No. of leafs etc.
5. At each step the best value for parameter will be outputed. Before moving to next parameter, set the best value of the current parameter in the model.
6. Prepare your best model in this way

In [15]:
# Define the range for the parameter
parameterRange = [10,20,50,100,200]
IS_Overall = []
OS_Overall = []
for nes in parameterRange :
    IS=[]
    OS=[]
    for i in range(1,20):
        # Don't cahnge anything here
        X_train, X_test, y_train, y_test = train_test_split(df_main[predictors], df_main[output], test_size=0.33, random_state=i*10)
        
        # Define your model here
        mod_rf = RandomForestRegressor(n_estimators = nes, # Keep this parameter variable as it is being optimized 
                                       oob_score = True, n_jobs = -1,random_state =50,
                                       max_features = "auto", min_samples_leaf = 2)
        
        # Don't change anything here
        X_train[scale_columns] = scaler.fit_transform(X_train[scale_columns])
        X_test[scale_columns] = scaler.transform(X_test[scale_columns])
        
        # Change the model name to your model
        mod_rf.fit(X_train, y_train)
        ypredTr = mod_rf.predict(X_train)
        ypredTs = mod_rf.predict(X_test)
        
        # Don't change anything here
        IS.append(r2_score(y_train, ypredTr))
        OS.append(r2_score(y_test, ypredTs))
    
    # Don't change anything here
    print("Parameter: ", nes, "IS :", np.mean(IS))
    print("Parameter: ", nes, "OS :", np.mean(OS))
    IS_Overall.append(np.mean(IS))
    OS_Overall.append(np.mean(OS))

# Don't change anything here 
print(" Best Paramter:", parameterRange[np.argmax(OS_Overall)]," IS R2:",IS_Overall[np.argmax(OS_Overall)]," OS R2:", OS_Overall[np.argmax(OS_Overall)])    

Parameter:  10 IS : 0.762364754565
Parameter:  10 OS : 0.322340977757
Parameter:  20 IS : 0.779338297019
Parameter:  20 OS : 0.344765477447
Parameter:  50 IS : 0.788676938807
Parameter:  50 OS : 0.357339199577
Parameter:  100 IS : 0.793344030462
Parameter:  100 OS : 0.361708347799
Parameter:  200 IS : 0.794925925676
Parameter:  200 OS : 0.365084703116
 Best Paramter: 200  IS R2: 0.794925925676  OS R2: 0.365084703116


## Part 3: Prepare the Output
In the following code just change your model and leave everything as it. 

In [67]:
# Define model here with all the best parameters
model = RandomForestRegressor(n_estimators = 200, oob_score = True, n_jobs = -1,random_state =50,
                                       max_features = "auto", min_samples_leaf = 1, max_depth=10)

### Don't change anything below this


df_mainDup = df_main.copy(deep=True)
df_features = pd.DataFrame(columns= predictors)
for i in range(4):
    np.random.seed(2016)
    Shuffle = list(np.random.permutation(len(df_main)))
    del Shuffle[int(i*len(df_main)/4):int((i+1)*(len(df_main))/4)]
    X_train = df_main.loc[Shuffle, predictors]
    y_train = df_main.loc[Shuffle, output]
    X_test = df_main.loc[:, predictors]
    y_test = df_main.loc[:, output]
    X_train[scale_columns] = scaler.fit_transform(X_train[scale_columns])
    X_test[scale_columns] = scaler.transform(X_test[scale_columns])
    model.fit(X_train, y_train)
    ypredTs = model.predict(X_test)
    df_mainDup['Pred'+str(i)] = ypredTs
    print("Total R2", r2_score(y_test, ypredTs))
    df_features.loc[i,:] = model.feature_importances_
    
df_mainDup.to_csv("20181214_HaomingOutput.csv")
df_features.to_csv("20181214_HaomingFeatures.csv")

Total R2 0.6798400685
Total R2 0.661185707107
Total R2 0.649002532167
Total R2 0.679450575536


## End: Please Download the Output File and Email to me.